In [1]:
import pandas as pd
import datetime 
from slugify import slugify

data_dir = '../data/'
now = datetime.datetime.now()

def __my_flatten_cols(self, how="_".join, reset_index=True):
    how = (lambda iter: list(iter)[-1]) if how == "last" else how
    self.columns = [how(filter(None, map(str, levels))) for levels in self.columns.values] \
                    if isinstance(self.columns, pd.MultiIndex) else self.columns
    return self.reset_index() if reset_index else self
pd.DataFrame.my_flatten_cols = __my_flatten_cols

In [2]:
#Mine-specific data
mine_data = pd.read_csv(data_dir + 'msha_mine_20190209-0.csv', escapechar='\\')
       
mine_rename = {'controller_id':'curr_ctrlr_id', 'controller_nm':'curr_ctrlr_nm', 
                'oper_id':'curr_oper_id', 'oper_nm':'curr_oper_nm'}
                
mine_data = mine_data.rename(index=str, columns=mine_rename)

In [3]:
#Controller-Operation relationship data
cntrlr_hist_data = pd.read_csv(data_dir + 'msha_controller_history_20190105-0.csv')
cntrlr_hist_data['ctrlr_end_dt'] = cntrlr_hist_data['ctrlr_end_dt'].fillna('2019-01-01')
cntrlr_hist_data['ctrlr_start_dt'] =  pd.to_datetime(cntrlr_hist_data['ctrlr_start_dt'], 
                                                      format='%Y-%m-%d')
cntrlr_hist_data['ctrlr_end_dt'] =  pd.to_datetime(cntrlr_hist_data['ctrlr_end_dt'], 
                                                      format='%Y-%m-%d')

In [4]:
def curr_mine_data_merge(mine_level_agg):
    
    keep_cols = ['mine_id','curr_mine_nm', 'c_m_ind', 'mine_type_cd', 'curr_stat_cd',
           'curr_stat_dt', 'curr_ctrlr_id', 'curr_ctrlr_nm', 'curr_oper_id',
           'curr_oper_nm', 'state_abbr', 'fips_cnty_nm', 'curr_ownr_beg_dt']
    mine_data_refined = mine_data[keep_cols]
    
    agg_mine = pd.merge(mine_level_agg, mine_data_refined, 
                        how='left', on='mine_id', suffixes=('_a','_m'))
    #agg_mine_del = pd.merge(agg_mine, mine_del_summary, 
    #                        how='left', on='mine_id', suffixes=('_am','_d'))

    return agg_mine

In [5]:
def catod_mine_data_merge(delinquency_data):
    catod_del_merge = pd.merge(cntrlr_hist_data,delinquency_data,how='left',left_on='oper_id',right_on='violator_id')

    #Filters for the filtering we'll be doing on catod_del_merge
    gt_dates = catod_del_merge['delinquent_date'] >= catod_del_merge['ctrlr_start_dt']
    lt_dates = catod_del_merge['delinquent_date'] <= catod_del_merge['ctrlr_end_dt']

    #So here's where we drop the records where the dates don't match up
    catod_filtered = catod_del_merge[(gt_dates) & (lt_dates)]

    #We're getting rid of columns so that we can drop duplicates and get back to our original
    #delinquency_data count
    catod_reduced = catod_filtered[['oper_id','ctrlr_id','ctrlr_start_dt','ctrlr_end_dt','oper_nm',
                                    'ass_case_nbr','curr_ctrlr_id','mine_id','delinquent_date',
                                    'ending_balance']].drop_duplicates()

    catod_reduced = catod_reduced.rename(index=str, columns={'oper_id':'catod_oper_id',
                                                             'ctrlr_id':'catod_ctrlr_id',
                                                             'ctrlr_start_dt':'catod_ctrlr_start_dt',
                                                             'ctrlr_end_dt':'catod_ctrlr_end_dt',
                                                             'oper_nm':'catod_oper_nm'})
    return catod_reduced

In [6]:
#Delinquency data
delinquency_data = pd.read_csv(data_dir + 'debtbyage_20181205_REFINED.csv')
delinquency_data['Delinquent Date'] =  pd.to_datetime(delinquency_data['Delinquent Date'], 
                                                      format='%m/%d/%Y')
delinquency_data['year'] = pd.DatetimeIndex(delinquency_data['Delinquent Date']).year
delinquency_data['month'] = pd.DatetimeIndex(delinquency_data['Delinquent Date']).month

#because this delinquency data is fucked, we're going to remove everything that
#isn't related to delinquencies and then do our catod and current joins
delinquency_data = delinquency_data[['Assess Case Nbr','Mine ID','Violator ID','Violator Name', 
                                     'Violator Type','Controller Id','Controller Name','Age Category',
                                     'Delinquent Date','Delinquent Type','Ending Balance','year','month']]

delinquency_data = delinquency_data.rename(index=str, columns={'Assess Case Nbr':'ass_case_nbr',
                                                              'Mine ID':'mine_id',
                                                              'Violator ID':'violator_id',
                                                              'Violator Name':'violator_nm',
                                                              'Controller Id':'dba_ctrlr_id',
                                                              'Controller Name':'dba_ctrlr_nm',
                                                              'Violator Type':'violator_type',
                                                              'Age Category':'age_cateogry',
                                                              'Delinquent Date':'delinquent_date',
                                                              'Delinquent Type':'delinquent_type',
                                                              'Ending Balance':'ending_balance'})

earliest_dict = delinquency_data.groupby('mine_id')['delinquent_date'].agg('min').to_dict()
earliest_series = delinquency_data.groupby('mine_id')['delinquent_date'].agg('min')
earliest_df = pd.DataFrame({'mine_id':earliest_series.index, 'earliest_date':earliest_series.values})  
#Need to round earliest_date up to the next year so we can factor only years with full delinquency
earliest_df['earliest_year'] = earliest_df['earliest_date'].map(lambda x: x.year+1)

mine_del_summary = delinquency_data.groupby('mine_id').agg({'delinquent_date':['max','min'],
                                                            'ending_balance':['sum','count']}).my_flatten_cols()
mine_del_summary = mine_del_summary.rename(index=str, columns={'delinquent_date_max':'latest_del_date',
                                                               'delinquent_date_min':'earliest_del_date',
                                                               'ending_balance_sum':'total_due',
                                                               'ending_balance_count':'num_del_records'})

del_data_curr_join = curr_mine_data_merge(delinquency_data)
del_data_curr_catod_reduced = catod_mine_data_merge(del_data_curr_join)
del_data_curr_catod = curr_mine_data_merge(del_data_curr_catod_reduced)

In [7]:
print(del_data_curr_catod.columns)

Index(['catod_oper_id', 'catod_ctrlr_id', 'catod_ctrlr_start_dt',
       'catod_ctrlr_end_dt', 'catod_oper_nm', 'ass_case_nbr',
       'curr_ctrlr_id_a', 'mine_id', 'delinquent_date', 'ending_balance',
       'curr_mine_nm', 'c_m_ind', 'mine_type_cd', 'curr_stat_cd',
       'curr_stat_dt', 'curr_ctrlr_id_m', 'curr_ctrlr_nm', 'curr_oper_id',
       'curr_oper_nm', 'state_abbr', 'fips_cnty_nm', 'curr_ownr_beg_dt'],
      dtype='object')


In [ ]:
justice_family_responsible = del_data_curr_catod[del_data_curr_catod['curr_ctrlr_id_a'].isin(['C04355','0091855'])]
justice_family_originated = del_data_curr_catod[del_data_curr_catod['catod_ctrlr_id'].isin(['C04355','0091855'])]
#justice_family_responsible.to_csv(data_dir + 'analysis/justice/20190325-justice-tester-curr-catod-v2.csv')
print(justice_family_responsible['ending_balance'].sum())
print(justice_family_originated['ending_balance'].sum())

## Controller Debt Analysis

In [10]:
#Debt That They are Currently Responsible For
def get_responsible_debt(ctrlr_id):
    return del_data_curr_catod[del_data_curr_catod['curr_ctrlr_id_m'].isin(ctrlr_id)]['ending_balance'].sum()


#Mine Count from Responsible Debt
def get_responsible_records(ctrlr_id):
    return del_data_curr_catod[del_data_curr_catod['curr_ctrlr_id_m'].isin(ctrlr_id)]['mine_id'].count()

def get_responsible_mines(ctrlr_id):
    return del_data_curr_catod[del_data_curr_catod['curr_ctrlr_id_m'].isin(ctrlr_id)]['mine_id'].nunique()


#Debt That They Originated
def get_originated_debt(ctrlr_id):
    return del_data_curr_catod[del_data_curr_catod['catod_ctrlr_id'].isin(ctrlr_id)]['ending_balance'].sum()


#Mine Count from Originated Debt
def get_originated_records(ctrlr_id):
    return del_data_curr_catod[del_data_curr_catod['catod_ctrlr_id'].isin(ctrlr_id)]['mine_id'].count()

def get_originated_mines(ctrlr_id):
    return del_data_curr_catod[del_data_curr_catod['catod_ctrlr_id'].isin(ctrlr_id)]['mine_id'].nunique()


#Debt That They Originated And Are Still Responsible For
def get_originated_responsible_debt(ctrlr_id):
    originated = del_data_curr_catod['catod_ctrlr_id'].isin(ctrlr_id)
    responsible = del_data_curr_catod['curr_ctrlr_id_m'].isin(ctrlr_id)
    return del_data_curr_catod[(originated) & (responsible)]['ending_balance'].sum()


#Debt That They Originated But Are NOT Still Responsible For
def get_originated_not_responsible_debt(ctrlr_id):
    originated = del_data_curr_catod['catod_ctrlr_id'].isin(ctrlr_id)
    responsible = del_data_curr_catod['curr_ctrlr_id_m'].isin(ctrlr_id)
    return del_data_curr_catod[(originated) & (~responsible)]['ending_balance'].sum()


#Debt That They Are Responsible For But Did Not Originate
def get_not_originated_responsible_debt(ctrlr_id):
    originated = del_data_curr_catod['catod_ctrlr_id'].isin(ctrlr_id)
    responsible = del_data_curr_catod['curr_ctrlr_id_m'].isin(ctrlr_id)
    return del_data_curr_catod[(~originated) & (responsible)]['ending_balance'].sum()


#Output the delinquent debt as a csv
def print_responsible_debt(ctrlr_id, ctrlr_nm):
    responsible_data = del_data_curr_catod[del_data_curr_catod['curr_ctrlr_id_m'].isin(ctrlr_id)]
    responsible_data.to_csv(data_dir + 'analysis/by_controller/' + slugify(ctrlr_nm) + '-responsible-delinquencies-NEW.csv')
    
def print_originated_debt(ctrlr_id, ctrlr_nm):
    originated_data = del_data_curr_catod[del_data_curr_catod['catod_ctrlr_id'].isin(ctrlr_id)]
    originated_data.to_csv(data_dir + 'analysis/by_controller/' + slugify(ctrlr_nm) + '-originated-delinquencies.csv')


regional_debtors = [{'cntrl_nm':'James C Justice II', 'cntrl_id':['C04355']},
                    {'cntrl_nm':'James C  Justice lll; Jillian L  Justice', 'cntrl_id':['0091855']},
                    {'cntrl_nm':'Justice family companies', 'cntrl_id':['0091855','C04355']},
                    {'cntrl_nm':'Jeffrey Hoops, Revelation Energy', 'cntrl_id':['C15863']},
                    {'cntrl_nm':'Virginia Conservation Legacy Fund/ERP Compliant','cntrl_id':['0121101']},
                    {'cntrl_nm':'James H Booth','cntrl_id':['C11194']}]

total_del_sum = del_data_curr_catod['ending_balance'].sum()

for cntrl in regional_debtors:
    ctrlr_id = cntrl['cntrl_id']
    ctrlr_nm = cntrl['cntrl_nm']
    responsible = '${:,.2f}'.format(get_responsible_debt(ctrlr_id))
    originated = '${:,.2f}'.format(get_originated_debt(ctrlr_id))
    originated_responsible = '${:,.2f}'.format(get_originated_responsible_debt(ctrlr_id))
    originated_not_responsible = '${:,.2f}'.format(get_originated_not_responsible_debt(ctrlr_id))
    not_originated_responsible = '${:,.2f}'.format(get_not_originated_responsible_debt(ctrlr_id))
    
    responsible_mines = get_responsible_mines(ctrlr_id)
    originated_mines = get_originated_mines(ctrlr_id)
    
    responsible_records = get_responsible_records(ctrlr_id)
    originated_records = get_originated_records(ctrlr_id)
    
    print_responsible_debt(ctrlr_id, ctrlr_nm)
    print_originated_debt(ctrlr_id, ctrlr_nm)
    
    print('------------------')
    print(ctrlr_nm)
    print('------------------')
    print('Responsible: ' + responsible)
    print('Originated: ' + originated)
    print('Originated & Responsible: ' + originated_responsible)
    print('Originated but NOT Responsible: ' + originated_not_responsible)
    print('Responsible but NOT Originated: ' + not_originated_responsible)
    print('')
    print('Unique mines with responsible debt: ' + str(responsible_mines))
    print('Unique mines with originated debt: ' + str(originated_mines))
    print('')
    print('Responsible debt records: ' + str(responsible_records))
    print('Originated debt records: ' + str(originated_records))
    print('')
    
    print('Details for controller: ' + ctrlr_nm + ' (' + '; '.join(ctrlr_id) + ')')
    print('When you were controller at ' + str(originated_mines) + ' mines you originated delinquent debt that now totals ' 
          + originated + '.')
    print('Some of those mines you still own, some have been sold. Our understanding is that debt remains with the mines.')
    print('According to our analysis of MSHA data, you currently control ' + str(responsible_mines) 
          + ' mines and are responsible for ' + responsible + ' in delinquent debt at those mines.')
    print('')

------------------
James C Justice II
------------------
Responsible: $1,145,089.76
Originated: $4,790,373.26
Originated & Responsible: $765,020.51
Originated but NOT Responsible: $4,025,352.75
Responsible but NOT Originated: $380,069.25

Unique mines with responsible debt: 15
Unique mines with originated debt: 74

Responsible debt records: 132
Originated debt records: 588

Details for controller: James C Justice II (C04355)
When you were controller at 74 mines you originated delinquent debt that now totals $4,790,373.26.
Some of those mines you still own, some have been sold. Our understanding is that debt remains with the mines.
According to our analysis of MSHA data, you currently control 15 mines and are responsible for $1,145,089.76 in delinquent debt at those mines.

------------------
James C  Justice lll; Jillian L  Justice
------------------
Responsible: $3,637,398.50
Originated: $857,058.60
Originated & Responsible: $544,675.30
Originated but NOT Responsible: $312,383.30
Resp